In [24]:
import os, sys
import numpy as np
from collections import OrderedDict as OD
import math
#import uproot3
import uproot as uproot
import hist
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
import enum
import mplhep as hep
from parse import *
import copy

sys.path.append( os.path.abspath('../') )
print(f"{os.path.abspath('../') = }")

from htoaa_CommonTools import (
    rebinTH1, rebinTH2, variableRebinTH1,
)

sIpFile = '/eos/cms/store/user/ssawant/htoaa/analysis/20240131_GGFMode_DataVsMC/2018/analyze_htoaa_stage1.root'
#sOpFile = '/eos/cms/store/user/ssawant/htoaa/analysis/20240131_GGFMode_DataVsMC/2018/analyze_htoaa_stage1_for2DAlphabet.root'
sOpDir  = '/eos/cms/store/user/ssawant/htoaa/analysis/20240131_GGFMode_DataVsMC/2018/2DAlphabet_inputFiles'
#sOpDir  = '/eos/cms/store/user/ssawant/htoaa/analysis/20240131_GGFMode_DataVsMC/2018/2DAlphabet_inputFiles_1'

fIpFile = uproot.open(sIpFile)
#fOpFile = uproot.recreate(sOpFile)
if not os.path.exists(sOpDir):
    os.makedirs(sOpDir)


os.path.abspath('../') = '/afs/cern.ch/work/s/ssawant/private/htoaa/htoaa_b_ana_SS'


In [25]:
processes_dict = {
    #'data_obs': ['JetHT_Run2018A', 'JetHT_Run2018B', 'JetHT_Run2018C', 'JetHT_Run2018D'],
    'Run2_data': ['JetHT_Run2018A', 'JetHT_Run2018B', 'JetHT_Run2018C', 'JetHT_Run2018D'],
    'QCD': ['QCD_0bCat', 'QCD_1bCat', 'QCD_2bCat', 'QCD_3bCat', 'QCD_4bCat', 'QCD_5bAndMoreCat'],
    'TT': ['TTToHadronic_powheg', 'TTToSemiLeptonic_powheg', 'TTTo2L2Nu_powheg', "SingleTop"],
    'Z': ['ZJetsToQQ_HT', "DYJets_M-50_Incl_NLO"],
    'W': ['WJetsToQQ_HT', 'WJetsToLNu_HT_LO'],
    'SUSY_GluGluH_01J_HToAATo4B_M-12_HPtAbv150': ['SUSY_GluGluH_01J_HToAATo4B_M-12_HPtAbv150'],
    'SUSY_GluGluH_01J_HToAATo4B_M-15_HPtAbv150': ['SUSY_GluGluH_01J_HToAATo4B_M-15_HPtAbv150'],
    'SUSY_GluGluH_01J_HToAATo4B_M-20_HPtAbv150': ['SUSY_GluGluH_01J_HToAATo4B_M-20_HPtAbv150'],
    'SUSY_GluGluH_01J_HToAATo4B_M-25_HPtAbv150': ['SUSY_GluGluH_01J_HToAATo4B_M-25_HPtAbv150'],
    'SUSY_GluGluH_01J_HToAATo4B_M-30_HPtAbv150': ['SUSY_GluGluH_01J_HToAATo4B_M-30_HPtAbv150'],
    'SUSY_GluGluH_01J_HToAATo4B_M-35_HPtAbv150': ['SUSY_GluGluH_01J_HToAATo4B_M-35_HPtAbv150'],
    'SUSY_GluGluH_01J_HToAATo4B_M-40_HPtAbv150': ['SUSY_GluGluH_01J_HToAATo4B_M-40_HPtAbv150'],
    'SUSY_GluGluH_01J_HToAATo4B_M-45_HPtAbv150': ['SUSY_GluGluH_01J_HToAATo4B_M-45_HPtAbv150'],
    'SUSY_GluGluH_01J_HToAATo4B_M-50_HPtAbv150': ['SUSY_GluGluH_01J_HToAATo4B_M-50_HPtAbv150'],
    'SUSY_GluGluH_01J_HToAATo4B_M-55_HPtAbv150': ['SUSY_GluGluH_01J_HToAATo4B_M-55_HPtAbv150'],
    'SUSY_GluGluH_01J_HToAATo4B_M-60_HPtAbv150': ['SUSY_GluGluH_01J_HToAATo4B_M-60_HPtAbv150'],
}
systematics_list = ['central']
systematics_forData = 'noweight'
selectionTags = ['SRWP40', 'SBWP80to40'] # ['SRWP40_mA30Window'] ['SRWP40'] ['SRWP40_mA55Window']  ['sel_leadingFatJetMSoftDrop', 'sel_leadingFatJetParticleNetMD_XbbvsQCD', 'SR'] #['SR', 'sel_leadingFatJetMSoftDrop', 'sel_leadingFatJetParticleNetMD_XbbvsQCD', 'sel_2018HEM1516Issue']
selectionTags_dict = {
    'Pass': 'SRWP40',
    'Fail': 'SBWP80to40'
}
histograms_list = [
    'hLeadingFatJetParticleNet_massH_Hto4b_avg_vs_massA_Hto4b_avg'
]
nRebinsX = 10
nRebinsY = 4

In [26]:
for processNameToUse, processNameList in processes_dict.items():
    sOpFile = '%s/%s.root' % (sOpDir, processNameToUse)
    fOpFile = uproot.recreate(sOpFile)

    for histo_name in histograms_list:
        for selectionTagNameToUse, selectionTagNameOriginal in selectionTags_dict.items():
            for systematic in systematics_list:
            
                hAdded = None
                for processName in processNameList:
                    systematicNameToUse = systematic 
                    if 'Run' in processName or 'ata' in processNameToUse:
                        systematicNameToUse = systematics_forData
                    histo_name_toUse_full = 'evt/%s/%s_%s_%s' % (processName, histo_name, selectionTagNameOriginal, systematicNameToUse)
                    print(f"{histo_name_toUse_full = }")
                    h = fIpFile[histo_name_toUse_full].to_hist()
                    h = h[::hist.rebin(nRebinsX), ::hist.rebin(nRebinsY)]
                    #print(f"After {h.axes = }, {h.axes[0] =  }")
                    if hAdded == None: hAdded = h
                    else:              hAdded = hAdded + h

                ## Set bins with nEvents < 0 to nEvents = 0
                nEvts          = hAdded.values()
                varNEvts       = hAdded.variances()                
                nEvts_modified = np.where(
                    (nEvts < 0),
                    np.full_like(nEvts, 1e-6),
                    nEvts
                )
                hAdded[:, :] = np.stack((nEvts_modified, varNEvts), axis=-1)
   
                '''
                #histoNameToSave = '%s/%s_%s_%s' % (processNameToUse, histo_name,selectionTagNameToUse,systematic)
                histoNameToSave = '%s/%s_%s' % (processNameToUse, histo_name,selectionTagNameToUse)
                print(f"{histoNameToSave = }")
                fOpFile[histoNameToSave] = hAdded
                '''

                histoNameToSave = '%s_%s' % (histo_name,selectionTagNameToUse)
                fOpFile[histoNameToSave] = hAdded

    fOpFile.close()




histo_name_toUse_full = 'evt/JetHT_Run2018A/hLeadingFatJetParticleNet_massH_Hto4b_avg_vs_massA_Hto4b_avg_SRWP40_noweight'
histo_name_toUse_full = 'evt/JetHT_Run2018B/hLeadingFatJetParticleNet_massH_Hto4b_avg_vs_massA_Hto4b_avg_SRWP40_noweight'
histo_name_toUse_full = 'evt/JetHT_Run2018C/hLeadingFatJetParticleNet_massH_Hto4b_avg_vs_massA_Hto4b_avg_SRWP40_noweight'
histo_name_toUse_full = 'evt/JetHT_Run2018D/hLeadingFatJetParticleNet_massH_Hto4b_avg_vs_massA_Hto4b_avg_SRWP40_noweight'
histo_name_toUse_full = 'evt/JetHT_Run2018A/hLeadingFatJetParticleNet_massH_Hto4b_avg_vs_massA_Hto4b_avg_SBWP80to40_noweight'
histo_name_toUse_full = 'evt/JetHT_Run2018B/hLeadingFatJetParticleNet_massH_Hto4b_avg_vs_massA_Hto4b_avg_SBWP80to40_noweight'
histo_name_toUse_full = 'evt/JetHT_Run2018C/hLeadingFatJetParticleNet_massH_Hto4b_avg_vs_massA_Hto4b_avg_SBWP80to40_noweight'
histo_name_toUse_full = 'evt/JetHT_Run2018D/hLeadingFatJetParticleNet_massH_Hto4b_avg_vs_massA_Hto4b_avg_SBWP80to40_noweight'
histo_na